### Data Engineering Pipeline: Zimbabwe Population-Specific Synthetic Data Transformation

# Transforming Bangladesh/South Asian maternal health datasets to Zimbabwe-specific context using mathematical modeling and Bayesian probability distributions from WHO statistics

In [55]:
!pip install matplotlib seaborn
!pip install sklearn
!pip install pandas
!pip install numpy
!pip install warnings
!pip install seaborn
!pip install scipy

  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'


  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 'scikit-learn' and report it to their issue tracker
      - as a last resort, set the environment variable
        SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
      
      More information is available at
      https://github.com/scikit-learn/sklearn-

ERROR: Could not find a version that satisfies the requirement warnings (from versions: none)
ERROR: No matching distribution found for warnings


In [57]:
# Importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.preprocessing import StandardScaler, LabelEncoder
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)

Analyzing Source Datasets

In [58]:
# Loading first dataset
df1 = pd.read_csv('datasets/maternal_health_features_dataset.csv')
print(f"\n Dataset 1 - Maternal Health Features (Birth Weight):")
print(f"Shape: {df1.shape}")
print(f"Features: {list(df1.columns)}")


 Dataset 1 - Maternal Health Features (Birth Weight):
Shape: (200, 15)
Features: ['age', 'pre_pregnancy_bmi', 'gestational_age_weeks', 'blood_pressure_systolic', 'blood_pressure_diastolic', 'hemoglobin_level', 'number_of_prenatal_visits', 'has_diabetes', 'has_hypertension', 'smoking_status', 'alcohol_consumption', 'education_level', 'household_income', 'iron_supplementation', 'birth_weight_category']


In [59]:
# Loading second dataset
df2 = pd.read_csv('datasets/full_maternal_health_risk_data.csv')
print(f"\n Dataset 2 - Full Maternal Health Risk:")
print(f"Shape: {df2.shape}")
print(f"Features: {list(df2.columns)}")


 Dataset 2 - Full Maternal Health Risk:
Shape: (1205, 12)
Features: ['Age', 'Systolic BP', 'Diastolic', 'BS', 'Body Temp', 'BMI', 'Previous Complications', 'Preexisting Diabetes', 'Gestational Diabetes', 'Mental Health', 'Heart Rate', 'Risk Level']


In [60]:
# Dataset 1 analysis
print(f"Sample size: {len(df1)}")
print(f"Missing values: {df1.isnull().sum().sum()}")
print(f"Age range: {df1['age'].min()}-{df1['age'].max()}")
print(f"Has hemoglobin: {'hemoglobin_level' in df1.columns}")
print(f"Has gestational age: {'gestational_age_weeks' in df1.columns}")

Sample size: 200
Missing values: 18
Age range: 18-44
Has hemoglobin: True
Has gestational age: True


In [61]:
# Dataset 2 analysis
print(f"Sample size: {len(df2)}")
print(f"Missing values: {df2.isnull().sum().sum()}")
print(f"Age range: {df2['Age'].min()}-{df2['Age'].max()}")
print(f"Risk distribution:")
for risk in df2['Risk Level'].value_counts().items():
    print(f"{risk[0]}: {risk[1]} ({risk[1]/len(df1)*100:.1f}%)")


Sample size: 1205
Missing values: 53
Age range: 10-325
Risk distribution:
Low: 713 (356.5%)
High: 474 (237.0%)


# Data preparation for merging

In [66]:
# Standardizing column names
df2_processed = df2.copy()
df2_processed.columns = df2_processed.columns.str.lower().str.replace(' ', '_')

# Renaming for consistency
df2_processed = df2_processed.rename(columns={
    'age': 'age',
    'systolic_bp': 'blood_pressure_systolic',
    'diastolic': 'blood_pressure_diastolic',
    'bs': 'blood_sugar',
    'body_temp': 'body_temperature',
    'bmi': 'bmi',
    'heart_rate': 'heart_rate',
    'risk_level': 'risk_level_original'
})

print(f"\n Standardized Dataset 2, df2_processed: {df2_processed.shape}, Features: {list(df2_processed.columns)}")



 Standardized Dataset 2, df2_processed: (1205, 12), Features: ['age', 'blood_pressure_systolic', 'blood_pressure_diastolic', 'blood_sugar', 'body_temperature', 'bmi', 'previous_complications', 'preexisting_diabetes', 'gestational_diabetes', 'mental_health', 'heart_rate', 'risk_level_original']


In [67]:
# Adding missing critical features from statistical imputation
# Feature 1: Hemoglobin (critical for Zimbabwe - 31% anemia)
# Imputing based on risk level and correlation with other vitals
def impute_hemoglobin(row):
    """
    Imputing hemoglobin based on clinical correlations
    Low hemoglobin correlates with: high risk, low BP, high HR
    """
    base_hb = 12.0  # Normal baseline
    
    # Adjusting for risk level
    if row['risk_level_original'] == 'High':
        base_hb -= 2.0  # High risk - likely anemic
    elif row['risk_level_original'] == 'Mid':
        base_hb -= 0.8
    
    # Adjusting for blood pressure (low BP may indicate anemia)
    if row['blood_pressure_systolic'] < 100:
        base_hb -= 0.5
    
    # Adjusting for heart rate (compensatory tachycardia in anemia)
    if row['heart_rate'] > 85:
        base_hb -= 0.3
    
    # Adding noise
    base_hb += np.random.normal(0, 0.5)
    
    return max(7.0, min(16.0, base_hb))

df2_processed['hemoglobin_level'] = df2_processed.apply(impute_hemoglobin, axis=1)
print(f"Hemoglobin imputed (mean: {df2_processed['hemoglobin_level'].mean():.2f} g/dL)")

# Feature 2: Gestational Age
df2_processed['gestational_age_weeks'] = np.random.choice(
    [35, 36, 37, 38, 39, 40], 
    size=len(df2_processed),
    p=[0.05, 0.10, 0.15, 0.25, 0.30, 0.15]  # Higher probability for term
)
print(f"Gestational age assigned (mean: {df2_processed['gestational_age_weeks'].mean():.1f} weeks)")

# Feature 3: Parity (inferred from age)
def assign_parity(age, risk):
    """Assigning parity based on age and risk factors"""
    if age < 20:
        return np.random.choice([0, 1], p=[0.7, 0.3])
    elif age < 25:
        return np.random.choice([0, 1, 2], p=[0.3, 0.5, 0.2])
    elif age < 30:
        return np.random.choice([1, 2, 3], p=[0.2, 0.5, 0.3])
    elif age < 35:
        return np.random.choice([2, 3, 4], p=[0.3, 0.4, 0.3])
    else:  # 35+
        return np.random.choice([3, 4, 5, 6], p=[0.3, 0.3, 0.2, 0.2])

df2_processed['parity'] = df2_processed.apply(
    lambda row: assign_parity(row['age'], row['risk_level_original']), axis=1
)
print(f"Parity assigned (mean: {df2_processed['parity'].mean():.1f})")

# Feature 4: Previous PPH & C-Section
df2_processed['previous_pph'] = (
    (df2_processed['previous_complications'] == 1) & 
    (np.random.random(len(df2_processed)) < 0.4)
).astype(int)

df2_processed['previous_csection'] = (
    (df2_processed['parity'] > 0) & 
    (np.random.random(len(df2_processed)) < 0.18)
).astype(int)

print(f"Previous PPH: {df2_processed['previous_pph'].sum()} cases")
print(f"Previous C-Section: {df2_processed['previous_csection'].sum()} cases")

# Feature 5: Prenatal visits
def assign_prenatal_visits(risk, age):
    """Assign prenatal visits based on risk and age"""
    if risk == 'High':
        return np.random.poisson(5)  # High risk - fewer visits (missed care)
    elif risk == 'Mid':
        return np.random.poisson(7)
    else:
        return np.random.poisson(8)

df2_processed['number_of_prenatal_visits'] = df2_processed.apply(
    lambda row: max(0, min(15, assign_prenatal_visits(row['risk_level_original'], row['age']))),
    axis=1
)
print(f"Prenatal visits assigned (mean: {df2_processed['number_of_prenatal_visits'].mean():.1f})")

df2_processed.drop(columns=["risk_level_original"], inplace=True)

print(f"\n Enhanced dataset shape: {df2_processed.shape}")

Hemoglobin imputed (mean: 11.12 g/dL)
Gestational age assigned (mean: 38.1 weeks)
Parity assigned (mean: 2.0)
Previous PPH: 78 cases
Previous C-Section: 186 cases
Prenatal visits assigned (mean: 6.8)

 Enhanced dataset shape: (1205, 17)


# Extracting Zimbabwe Statistics from WHO Data

In [69]:
# Based on WHO statistics
ZIMBABWE_STATS = {
    # Demographics
    'anemia_prevalence': 0.31,  # 31% (WHO-aligned)
    'maternal_age_mean': 28.5,
    'maternal_age_std': 6.5,
    'advanced_maternal_age_rate': 0.28,  # >35 years
    
    # Physiological
    'baseline_hemoglobin_mean': 11.2,  # g/dL (accounting for anemia)
    'baseline_hemoglobin_std': 1.4,
    'anemic_hemoglobin_mean': 9.5,
    'anemic_hemoglobin_std': 0.8,
    
    # BMI
    'bmi_mean': 24.8,
    'bmi_std': 4.5,
    'obesity_rate': 0.15,  # BMI > 30
    
    # Parity
    'parity_mean': 2.8,  # Higher than Western cohorts
    'parity_lambda': 2.3,  # Poisson distribution parameter
    
    # Blood Pressure (higher baseline stress)
    'sbp_mean': 118,
    'sbp_std': 15,
    'dbp_mean': 75,
    'dbp_std': 11,
    'hypertension_rate': 0.22,  # Higher than Bangladesh
    
    # Heart Rate (compensatory mechanisms)
    'hr_mean': 78,
    'hr_std': 9,
    
    # Diabetes
    'diabetes_prevalence': 0.08,  # Lower than South Asia
    'gestational_diabetes_rate': 0.06,
    
    # Prenatal Care
    'inadequate_care_rate': 0.35,  # <4 visits
    'average_visits': 5.2,
    
    # Gestational Age
    'preterm_rate': 0.18,  # <37 weeks
    'term_mean': 38.2,
}

print("\n ZIMBABWE BASELINE PARAMETERS:")
for key, value in ZIMBABWE_STATS.items():
    if isinstance(value, float) and value < 1:
        print(f"{key:30s}: {value*100:5.1f}%")
    else:
        print(f"{key:30s}: {value:5.2f}")


 ZIMBABWE BASELINE PARAMETERS:
anemia_prevalence             :  31.0%
maternal_age_mean             : 28.50
maternal_age_std              :  6.50
advanced_maternal_age_rate    :  28.0%
baseline_hemoglobin_mean      : 11.20
baseline_hemoglobin_std       :  1.40
anemic_hemoglobin_mean        :  9.50
anemic_hemoglobin_std         :  80.0%
bmi_mean                      : 24.80
bmi_std                       :  4.50
obesity_rate                  :  15.0%
parity_mean                   :  2.80
parity_lambda                 :  2.30
sbp_mean                      : 118.00
sbp_std                       : 15.00
dbp_mean                      : 75.00
dbp_std                       : 11.00
hypertension_rate             :  22.0%
hr_mean                       : 78.00
hr_std                        :  9.00
diabetes_prevalence           :   8.0%
gestational_diabetes_rate     :   6.0%
inadequate_care_rate          :  35.0%
average_visits                :  5.20
preterm_rate                  :  18.0%
term_mea

# Bayesian Transformation To Zimbabwe Context (Bangladesh(South Asia) - Zimbabwe(Sub-Saharan Africa))

In [71]:
# Creating Zimbabwe cohort
df_zimbabwe = df2_processed.copy()

print("\n Applying Population-Specific Transformations...")

# Transformation 1: Age Distribution Adjustment
print("\n1. Age Distribution Transformation:")
current_age_mean = df_zimbabwe['age'].mean()
current_age_std = df_zimbabwe['age'].std()
print(f"Bangladesh: μ={current_age_mean:.1f}, σ={current_age_std:.1f}")
print(f"Zimbabwe Target: μ={ZIMBABWE_STATS['maternal_age_mean']:.1f}, σ={ZIMBABWE_STATS['maternal_age_std']:.1f}")

# Applying z-score normalization and rescale
df_zimbabwe['age'] = (
    (df_zimbabwe['age'] - current_age_mean) / current_age_std * 
    ZIMBABWE_STATS['maternal_age_std'] + 
    ZIMBABWE_STATS['maternal_age_mean']
)
df_zimbabwe['age'] = df_zimbabwe['age'].clip(15, 50).round().astype(int)
print(f"Transformed: μ={df_zimbabwe['age'].mean():.1f}, σ={df_zimbabwe['age'].std():.1f}")

# Transformation 2: Hemoglobin - Critical for Zimbabwe (31% anemia)
print("\n2. Hemoglobin Distribution (CRITICAL - 31% Anemia Prevalence):")
current_anemia_rate = (df_zimbabwe['hemoglobin_level'] < 11.0).mean()
print(f"Current anemia rate: {current_anemia_rate*100:.1f}%")
print(f"Target anemia rate: {ZIMBABWE_STATS['anemia_prevalence']*100:.1f}%")

# Bayesian adjustment to match 31% anemia prevalence
n_total = len(df_zimbabwe)
n_anemic_target = int(n_total * ZIMBABWE_STATS['anemia_prevalence'])
n_anemic_current = (df_zimbabwe['hemoglobin_level'] < 11.0).sum()

if n_anemic_current < n_anemic_target:
    non_anemic_indices = df_zimbabwe[df_zimbabwe['hemoglobin_level'] >= 11.0].index
    to_convert = np.random.choice(
        non_anemic_indices,
        size=n_anemic_target - n_anemic_current,
        replace=False
    )
    df_zimbabwe.loc[to_convert, 'hemoglobin_level'] = np.random.normal(
        ZIMBABWE_STATS['anemic_hemoglobin_mean'],
        ZIMBABWE_STATS['anemic_hemoglobin_std'],
        size=len(to_convert)
    )
else:
    anemic_indices = df_zimbabwe[df_zimbabwe['hemoglobin_level'] < 11.0].index
    to_convert = np.random.choice(
        anemic_indices,
        size=n_anemic_current - n_anemic_target,
        replace=False
    )
    df_zimbabwe.loc[to_convert, 'hemoglobin_level'] = np.random.normal(
        ZIMBABWE_STATS['baseline_hemoglobin_mean'],
        ZIMBABWE_STATS['baseline_hemoglobin_std'],
        size=len(to_convert)
    )

# Ensuring all values are realistic
df_zimbabwe['hemoglobin_level'] = df_zimbabwe['hemoglobin_level'].clip(6.5, 16.0)
final_anemia_rate = (df_zimbabwe['hemoglobin_level'] < 11.0).mean()
print(f"Achieved anemia rate: {final_anemia_rate*100:.1f}%")
print(f"Mean Hb: {df_zimbabwe['hemoglobin_level'].mean():.2f} g/dL")

# Transformation 3: BMI Adjustment
print("\n3. BMI Distribution Transformation:")
current_bmi_mean = df_zimbabwe['bmi'].mean()
df_zimbabwe['bmi'] = df_zimbabwe['bmi'].fillna(
    np.random.normal(ZIMBABWE_STATS['bmi_mean'], ZIMBABWE_STATS['bmi_std'])
)
# Adjust distribution
df_zimbabwe['bmi'] = (
    (df_zimbabwe['bmi'] - current_bmi_mean) / df_zimbabwe['bmi'].std() * 
    ZIMBABWE_STATS['bmi_std'] + 
    ZIMBABWE_STATS['bmi_mean']
)
df_zimbabwe['bmi'] = df_zimbabwe['bmi'].clip(14, 42).round(1)
print(f"Transformed: μ={df_zimbabwe['bmi'].mean():.1f}, σ={df_zimbabwe['bmi'].std():.1f}")

# Transformation 4: Blood Pressure (Higher baseline in SSA due to stress)
print("\n4. Blood Pressure Adjustment (SSA context - higher baseline stress):")
# Systolic - fill NaN first
df_zimbabwe['blood_pressure_systolic'] = df_zimbabwe['blood_pressure_systolic'].fillna(
    np.random.normal(ZIMBABWE_STATS['sbp_mean'], ZIMBABWE_STATS['sbp_std'])
)
df_zimbabwe['blood_pressure_systolic'] = (
    df_zimbabwe['blood_pressure_systolic'] + 
    np.random.normal(8, 3, len(df_zimbabwe))
).clip(85, 180).round().astype(int)

# Diastolic - fill NaN first
df_zimbabwe['blood_pressure_diastolic'] = df_zimbabwe['blood_pressure_diastolic'].fillna(
    np.random.normal(ZIMBABWE_STATS['dbp_mean'], ZIMBABWE_STATS['dbp_std'])
)
df_zimbabwe['blood_pressure_diastolic'] = (
    df_zimbabwe['blood_pressure_diastolic'] + 
    np.random.normal(5, 2, len(df_zimbabwe))
).clip(50, 120).round().astype(int)

print(f"SBP: μ={df_zimbabwe['blood_pressure_systolic'].mean():.1f}, σ={df_zimbabwe['blood_pressure_systolic'].std():.1f}")
print(f"DBP: μ={df_zimbabwe['blood_pressure_diastolic'].mean():.1f}, σ={df_zimbabwe['blood_pressure_diastolic'].std():.1f}")

# Transformation 5: Parity (Higher in Zimbabwe)
print("\n5. Parity Distribution (Higher in Sub-Saharan Africa):")
df_zimbabwe['parity'] = np.random.poisson(ZIMBABWE_STATS['parity_lambda'], len(df_zimbabwe))
df_zimbabwe['parity'] = df_zimbabwe['parity'].clip(0, 8)
print(f"Mean parity: {df_zimbabwe['parity'].mean():.2f}")
print(f"High parity (>3): {(df_zimbabwe['parity'] > 3).mean()*100:.1f}%")

# Transformation 6: Diabetes (Lower in Zimbabwe vs South Asia)
print("\n6. Diabetes Prevalence Adjustment (Lower in SSA):")
current_diabetes_rate = df_zimbabwe['preexisting_diabetes'].mean()
print(f"Bangladesh diabetes rate: {current_diabetes_rate*100:.1f}%")
print(f"Zimbabwe target rate: {ZIMBABWE_STATS['diabetes_prevalence']*100:.1f}%")

# Reducing diabetes prevalence
n_diabetes_current = df_zimbabwe['preexisting_diabetes'].sum()
n_diabetes_target = int(len(df_zimbabwe) * ZIMBABWE_STATS['diabetes_prevalence'])

if n_diabetes_current > n_diabetes_target:
    diabetic_indices = df_zimbabwe[df_zimbabwe['preexisting_diabetes'] == 1].index
    to_remove = np.random.choice(
        diabetic_indices,
        size=int(n_diabetes_current - n_diabetes_target),
        replace=False
    )
    df_zimbabwe.loc[to_remove, 'preexisting_diabetes'] = 0

print(f"Achieved diabetes rate: {df_zimbabwe['preexisting_diabetes'].mean()*100:.1f}%")

# Transformation 7: Prenatal Care (Lower in Zimbabwe)
print("\n7. Prenatal Care Access Adjustment:")
df_zimbabwe['number_of_prenatal_visits'] = np.random.poisson(
    ZIMBABWE_STATS['average_visits'], 
    len(df_zimbabwe)
).clip(0, 15)
inadequate_care_rate = (df_zimbabwe['number_of_prenatal_visits'] < 4).mean()
print(f"Average visits: {df_zimbabwe['number_of_prenatal_visits'].mean():.1f}")
print(f"Inadequate care (<4 visits): {inadequate_care_rate*100:.1f}%")


 Applying Population-Specific Transformations...

1. Age Distribution Transformation:
Bangladesh: μ=27.7, σ=12.6
Zimbabwe Target: μ=28.5, σ=6.5
Transformed: μ=28.5, σ=4.8

2. Hemoglobin Distribution (CRITICAL - 31% Anemia Prevalence):
Current anemia rate: 41.1%
Target anemia rate: 31.0%
Achieved anemia rate: 35.5%
Mean Hb: 11.26 g/dL

3. BMI Distribution Transformation:
Transformed: μ=24.8, σ=4.4

4. Blood Pressure Adjustment (SSA context - higher baseline stress):
SBP: μ=124.8, σ=18.6
DBP: μ=82.1, σ=14.3

5. Parity Distribution (Higher in Sub-Saharan Africa):
Mean parity: 2.32
High parity (>3): 21.1%

6. Diabetes Prevalence Adjustment (Lower in SSA):
Bangladesh diabetes rate: 28.8%
Zimbabwe target rate: 8.0%
Achieved diabetes rate: 8.0%

7. Prenatal Care Access Adjustment:
Average visits: 5.2
Inadequate care (<4 visits): 22.8%
